In [8]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

Fonctions pour nettoyer les datas

In [6]:
import pandas as pd
import numpy as np

In [13]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.join_data import *
from Tool_Functions.test_comportment_reabo import *
from Tool_Functions.visual import *
from Tool_Functions.comportment_reabo import *

In [14]:
def upload_clean(data_path):
    """
    Download clean files in your data_path
    There has to be the dirty files in your data_path
    """
    df = file_to_dataframe(data_path + "Correspondances_Promos.csv",";")
    df_Correspondances_Promos = change_dates_all(df,['DEBVAL', 'FINVAL', 'DEBABOMIN', 'DEBABOMAX'])
    save_to_csv_file(df_Correspondances_Promos,data_path + "df_Correspondances_Promos.csv")

    for i in range(1, 4, 1):
        df = file_to_dataframe(data_path + f"Donnees_Promos_202{i}.csv",",")
        df_Donnees_Promos_202i = clean_dates(df)
        save_to_csv_file(df_Donnees_Promos_202i, data_path + f"df_Donnees_Promos_202{i}.csv")

        df = file_to_dataframe(data_path + f"Donnees_Reabos_202{i}.csv",",")
        df_Donnees_Reabos_202i = clean_dates(df)
        save_to_csv_file(df_Donnees_Reabos_202i,data_path + f"df_Donnees_Reabos_202{i}.csv")

    return

def concat_all_years(data_path):
    """
    Create new csv files that concats the 3 years at once.
    """
    for name in ["df_Donnees_Promos_202", "df_Donnees_Reabos_202"]:
        df1 = file_to_dataframe(data_path + name + "1.csv",",")
        df2 = file_to_dataframe(data_path + name + "2.csv",",")
        df3 = file_to_dataframe(data_path + name + "3.csv",",")

        df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

        df.to_csv(data_path + name[:-4] + ".csv", index = True)


    return

upload_clean(path_antoine)
concat_all_years(path_antoine)

Partie sur les ODD 

In [11]:
def creation_df_odd(data_path, data_path_results):
    """
    This function creates the new df_odd on your Computer where all the ODD are presented and classified with their type
    """

    df_Correspondances_Promos = file_to_dataframe(data_path + "df_Correspondances_Promos.csv",",") 

    df_odd = df_filter_condition(df_Correspondances_Promos,'TYPE_PROMO','ODD') #we create a DataFrame with only ODD Promotion
    df_odd['TYPE_PROMON'] = create_new_column(df_odd,apply_conditions) #we create new columns on this DataFrame of the ODD type


    for i in [1, 2, 3]:
    #creation df_odd_202i pour chaque année
        df_Donnees_Promos_202i = file_to_dataframe(data_path + f"df_Donnees_Promos_202{i}.csv",",")

        n = df_Donnees_Promos_202i.shape[0] / 10000 #number minimum of used
        df_new_odd = keep_used_odd(df_Donnees_Promos_202i,df_odd,n) #creation of the new tab by keeping only the used promos
        
        save_to_csv_file(df_new_odd,data_path_results + f"odd_202{i}.csv")
    
    #idem sur les trois années
    df_Donnees_Promos = file_to_dataframe(data_path + "df_Donnees_Promos.csv",",")

    n = df_Donnees_Promos.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Donnees_Promos,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,data_path_results + f"odd.csv")

    return True


#Intermediary step we change the df_Données_Promos by adding a column 'TYPE_PROMON' for being easier to understand instead of 'CPROMO'

def create_df_Données_Promos_odd(data_path, data_path_results):


    for i in [1, 2, 3]:
        #année i
        df_Donnees_Promos_202i = file_to_dataframe(data_path + f"df_Donnees_Promos_202{i}.csv",",")
        df_odd = file_to_dataframe(data_path + f"odd_202{i}.csv", ",")
        df_Donnees_Promos_202i_odd = join_dataFrames(df_Donnees_Promos_202i,df_odd[['CPROMO','TYPE_PROMON']] ,'CPROMO') #We create a new column 'TYPE_PROMON' on df_Données_Promos_202i
        save_to_csv_file(df_Donnees_Promos_202i_odd,data_path_results + f"df_Donnees_Promos_202{i}_odd.csv") #we save it on your Mac

    return True

def create_df_Données_Promos_odd_all(data_path, data_path_results):
    """
    This function create df_Données_Promos_odd for the dataFrame with all years
    """
    
    df_Donnees_Promos = file_to_dataframe(data_path + "df_Donnees_Promos.csv",",")
    df_odd = file_to_dataframe(data_path + "odd.csv", ",")
    df_Donnees_Promos_odd = join_dataFrames(df_Donnees_Promos,df_odd[['CPROMO','TYPE_PROMON']] ,'CPROMO') #We create a new column 'TYPE_PROMON' on df_Données_Promos_202i
    save_to_csv_file(df_Donnees_Promos_odd,data_path_results + "df_Donnees_Promos_odd.csv") #we save it on your Mac
    
    return True

#Intermediary step we create df_Données_Réabos_odd where there are all the Reabos which corresponds to a reabo

def create_df_Données_Reabos_odd_all(data_path, data_path_results):
    """
    This function create df_Données_Reabos_odd with all years of Reabos which corresponds to a use of Promo
    and then we drop some unused column
    """
    df_Donnees_Promos_odd = file_to_dataframe(data_path +"df_Donnees_Promos_odd.csv" )
    df_Donnees_Reabos = file_to_dataframe(data_path + "df_Donnees_Reabos.csv")
    df_Donnees_Reabos_odd = join_dataFrames(df_Donnees_Promos_odd,df_Donnees_Reabos,['ID_ABONNE','DATE_ACTE_REEL'])

    df_Donnees_Reabos_odd = df_Donnees_Reabos_odd.drop(columns = ["REABO_APRES_ECHEANCE","CPROMO","SECTEUR","PAYS","NUMDIST_PARTENAIRE","NOM_PARTENAIRE","NUMDIST_POINT_DE_VENTE","NOM_POINT_DE_VENTE"])

    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Donnees_Reabos_odd = time_reabo_columns(df_Donnees_Reabos_odd,end_abo,date_reabo)
    
    save_to_csv_file(df_Donnees_Reabos_odd,data_path_results + "df_Donnees_Reabos_odd.csv")

    return True

creation_df_odd(path_antoine,path_results_antoine)
create_df_Données_Promos_odd(path_antoine, path_results_antoine)
create_df_Données_Promos_odd_all(path_antoine,path_results_antoine)
create_df_Données_Reabos_odd_all(path_antoine,path_results_antoine)